### Closure

In the blogosphere, closures are sometimes confused with anonymous functions. The
reason why many confuse them is historic: defining functions inside functions is not so
common, until you start using anonymous functions. And closures only matter when
you have nested functions. So a lot of people learn both concepts at the same time.

+  it can access nonglobal variables that are defined outside of its body.

In [3]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [4]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


###### inspecting the function created by *make_averager*

In [7]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value', 'total')
('series',)


In [8]:
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

(<cell at 0x0000000004BE8E88: list object at 0x0000000004C79188>,)
[10, 11, 12]


### *nonlocal* keyword

In [9]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

### useful decorator example: clock the function time

In [16]:
import time
def clock(func):
    def clocked(*args):
        t = time.perf_counter()
        result = func(*args)
        elasped = time.perf_counter() -t
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elasped, name, arg_str, result))
        return result
    return clocked

In [18]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n* factorial(n-1)

In [19]:
print("*" * 40, "calling snooze(.123)")
snooze(.123)
print('*' * 40, 'calling factorial(6)')
print('6! =', factorial(6))

**************************************** calling snooze(.123)
[0.12463369s] snooze(0.123) -> None
**************************************** calling factorial(6)
[0.00000200s] factorial(1) -> 1
[0.00015253s] factorial(2) -> 2
[0.00021924s] factorial(3) -> 6
[0.00028395s] factorial(4) -> 24
[0.00033926s] factorial(5) -> 120
[0.00039771s] factorial(6) -> 720
6! = 720


In [20]:
factorial.__name__

'clocked'

### how it works
+ records the initial time 
+ calls the original *factorial*, saving the result
+ computes the elapsed time
+ formats and prints the collected data
+ returns the result saved in step 2

This is the typical behavior of a decorator: it replaces the decorated function with a new
function that accepts the same arguments and (usually) returns whatever the decorated
function was supposed to return, while also doing some extra processing

###### An improved clock decorator

In [27]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked
        

###### Memoization with functools.lru_cache
The letters LRU stand for Least Recently Used, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while.

In [28]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [33]:
print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00000000s] fibonacci(6) -> 8
8


In [34]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

This is an example of stacked decorators: @lru_cache() is applied on the function returned by @clock.

the full signature is: *functools.lru_cache(maxsize=128, typed=False)*

In [35]:
print(fibonacci(10))

[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(2) -> 1
[0.00000000s] fibonacci(3) -> 2
[0.00000000s] fibonacci(4) -> 3
[0.00000000s] fibonacci(5) -> 5
[0.00000000s] fibonacci(6) -> 8
[0.00000000s] fibonacci(7) -> 13
[0.00000000s] fibonacci(8) -> 21
[0.00000000s] fibonacci(9) -> 34
[0.00000000s] fibonacci(10) -> 55
55


### Generic Functions with Single Dispatch
a solution in Python to handle function overload

When possible, register the specialized functions to handle ABCs (abstract classes) such
as numbers.Integral and abc.MutableSequence instead of concrete implementations
like int and list. This allows your code to support a greater variety of compatible types.
For example, a Python extension can provide alternatives to the int type with fixed bit
lengths as subclasses of numbers.Integral.

In [38]:
from functools import singledispatch
from collections import abc
import numbers
import html


# @singledispatch marks the base function that handles the object type.
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)


# The name of the specialized functions is irrelevant; _ is a good choice to make
# this clear
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace("\n", '<br>\n')
    return '<p>{0}</p>'.format(content)


# For each additional type to receive special treatment, register a new function.
# numbers.Integral is a virtual superclass of int.
@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)


# You can stack several register decorators to support different types with the
# same function
@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\m</ul>'

### Stacked Decorators

In [ ]:
@d1
@d2
def f():
   print('f')

# the same as 
def f():
    print('f')
f = d1(d2(f))

### Parameterized Dcorators
When parsing a decorator in source code, Python takes the decorated function and
passes it as the first argument to the decorator function. So how do you make a decorator
accept other arguments? The answer is: make a decorator factory that takes those arguments and returns a decorator, which is then applied to the function to be decorated.

In [40]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x0000000004F05E18>)
running main()
registry -> [<function f1 at 0x0000000004F05E18>]
running f1()


In [41]:
registry = set()
def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func))
        
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x0000000004F051E0>)
running register(active=True)->decorate(<function f2 at 0x0000000004F05D90>)


### the parameterized clock decorator

In [51]:
import time

# passed a dict to format, only whose match will be used.
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'


def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

In [50]:
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.13480520s] snooze(0.123) -> None
[0.12479997s] snooze(0.123) -> None
[0.12320113s] snooze(0.123) -> None


In [53]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.133s
snooze(0.123) dt=0.125s
snooze(0.123) dt=0.123s
